In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import numpy as np
import xarray as xr
import xclim
from distributed import Client

from ocr.utils import load_conus404, prep_encoding
from ocr.wind import classify_wind_directions, direction_histogram, nws_fire_weather

In [ ]:
client = Client()
client

In [ ]:
# Make relative humidity intermediate data variable

In [ ]:
recreate_rh = False
if recreate_rh:
    ds = load_conus404('T2')
    ds['TD2'] = load_conus404('TD2')['TD2']
    hurs = xclim.indicators.atmos.relative_humidity_from_dewpoint(tas=ds['T2'], tdps=ds['TD2'])
    hurs = dask.optimize(hurs)[0]
    hurs.to_zarr(
        's3://carbonplan-ocr/intermediate/met-data/conus404/hurs.zarr', consolidated=True, mode='w'
    )

In [ ]:
# Make sfcWindspeed and sfcWindfromdir intermediate variables

In [ ]:
recreate_winds = False
if recreate_winds:
    ds = load_conus404('U10')
    ds['V10'] = load_conus404('V10')['V10']
    winds = xclim.indicators.atmos.wind_speed_from_vector(uas=ds['U10'], vas=ds['V10'])[
        0
    ].to_dataset()
    winds['sfcWindfromdir'] = xclim.indicators.atmos.wind_speed_from_vector(
        uas=ds['U10'], vas=ds['V10']
    )[1]
    winds = dask.optimize(winds)[0]
    winds.to_zarr(
        's3://carbonplan-ocr/intermediate/met-data/conus404/winds.zarr', consolidated=True, mode='w'
    )

In [ ]:
# calculate whether there is fire weather

In [ ]:
hurs = xr.open_zarr('s3://carbonplan-ocr/intermediate/met-data/conus404/hurs.zarr')
wind = xr.open_zarr('s3://carbonplan-ocr/intermediate/met-data/conus404/winds.zarr')
fire_weather_mask = nws_fire_weather(
    hurs['hurs'],
    15,
    # reason that wind gusts are typically ~40% higher than average wind speed
    # and we want to base this on wind gusts (need a citation for this)
    wind['sfcWind'] * 1.4,
    35,
)

In [ ]:
fire_weather_mask = dask.optimize(fire_weather_mask)[0]

In [ ]:
# classify the winds into 8 cardinal directions
direction_indices = classify_wind_directions(wind['sfcWindfromdir'].where(fire_weather_mask))

In [ ]:
# create the distribution of winds based upon the classified wind directions
# if there are nans in the mix, they won't contribute to the distribution
fraction = xr.apply_ufunc(
    direction_histogram,
    direction_indices,
    input_core_dims=[['time']],
    output_core_dims=[['wind_direction']],
    output_sizes={'wind_direction': 8},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    kwargs={},
)

In [ ]:
fraction = fraction.assign_coords(wind_direction=np.arange(0, 8))  # .chunk({'x': 500, 'y': 500})
fraction = dask.optimize(fraction)[0]

In [ ]:
encoding = prep_encoding(fraction.to_dataset())

In [ ]:
fraction.to_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_distribution.zarr',
    zarr_format=3,
    encoding=encoding,
    mode='w',
    consolidated=True,
)

In [ ]:
fraction = xr.open_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_distribution.zarr'
)
# sum up the fractions of all of the wind directions. if any pixel sums to zero it will
# fall into the no_fire_weather mask and be cast to a `nan`. if any pixel has any fraction
# of timestamps with fire weather then it will be cast into the mask where there *is* fire weather
no_fire_weather = fraction.sum(dim='wind_direction')
mode = fraction.argmax(dim='wind_direction').where(no_fire_weather).chunk({'x': -1, 'y': -1})

encoding = prep_encoding(mode.to_dataset())

mode.to_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_mode.zarr',
    zarr_format=3,
    encoding=encoding,
    mode='w',
    consolidated=True,
)

In [ ]:
test = xr.open_zarr(
    's3://carbonplan-ocr/intermediate/met-data/conus404/fire_weather_wind_mode.zarr'
)

In [ ]:
test